In [1]:
import os,importlib,numpy,json
import math
import random
import statistics
import seaborn as sns
import json 
from sklearn import linear_model


import scipy,scipy.stats

#import miner_20190520 as miner

import pandas
pandas.options.display.max_rows=100
pandas.set_option('display.max_colwidth',-1)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.pyplot.rcParams['axes.axisbelow']=True
matplotlib.rcParams.update({'font.size':18,'font.family':'Arial','xtick.labelsize':20,'ytick.labelsize':20,'axes.labelsize':30})
matplotlib.rcParams['pdf.fonttype']=42

In [2]:
# organize programs + states into a list. ith element are the regulons in the (i+1)th programs
with open("transcriptional_programs.json") as TP:
    prog = json.load(TP)
programs=[]
for i in range(1,141):
    I=str(i)
    programs.append(prog[I])
    
with open("transcriptional_states.json") as TS:
    stat = json.load(TS)
states=[]
for i in range(1,len(stat)):
    I=str(i)
    states.append(stat[I])
    
expData = pandas.read_csv('MMRF_CoMMpass_IA9_E74GTF_Salmon_Gene_TPM.txt', sep="\s+", header=0, index_col=0)
regulonDf=pandas.read_csv("regulonDf.csv", header=0, index_col=0)
patients=list(expData.columns)

In [4]:
#THIS FRAME WILL TAKE THE STATES, COMPUTE THE AVERAGE ACROSS STATES AND ORDER THEM!
#get genes in columns. ignore the ERC ones (for now?)
cols=expData.columns.values.tolist()
statesUnAv=[]
for x in range(len(states)):
    statesUnAvInt=[list(expData.loc['ENSG00000238602':,states[x][y]]) for y in range(len(states[x])) if states[x][y] in cols]
    statesUnAv.append(statesUnAvInt)
    
#normalize each patient by it's max before taking the average
statesUnAvNorm=[]
for x in range(len(statesUnAv)):
    statesNormInt=[]
    for y in range(len(statesUnAv[x])):
        statArray=numpy.asarray(statesUnAv[x][y])#/(max(numpy.asarray(statesUnAv[x][y])))
        statesNormInt.append(list(statArray))
    statesUnAvNorm.append(statesNormInt)  
    
#compute average gene expression. put statesUnAv[i] in an array, transpose and compute average of each row for each i.
stateAv=[]
stateAv2=[]
stateSD=[]
stateSD2=[]
for x in range(len(statesUnAvNorm)):
    statesArray=numpy.asarray(statesUnAvNorm[x]).transpose()
    stateAvInt=[[x,numpy.average(statesArray[x])] for x in range(len(statesArray))]
    stateAvInt2=[numpy.average(statesArray[x]) for x in range(len(statesArray))]
    stateSDInt=[[x,numpy.std(statesArray[x])] for x in range(len(statesArray))]
    stateSDInt2=[numpy.std(statesArray[x]) for x in range(len(statesArray))]
    stateSD.append(stateSDInt)
    stateSD2.append(stateSDInt2)
    stateAv.append(stateAvInt)
    stateAv2.append(stateAvInt2)
def sortSecond(val): 
    return val[1]

#then sort them!
for x in range(len(stateAv)):
    stateAv[x].sort(key=sortSecond,reverse=True)
    
stateOrd=[]
stateAvOrd=[]
for x in range(len(stateAv)):
    stateArrayOrd=numpy.asarray(stateAv[x]).transpose()
    stateOrd.append(list(stateArrayOrd[0]))
    stateAvOrd.append(list(stateArrayOrd[1]))
    
#stateOrd contains the gene ranks ordered by TPM 
#and labeled by their row in the expData frame starting with the ENGS genes
#difficulty is dealing with degenerate TPMs. zero expresisons will get mapped to itself.

stateStd=[]
stateStdOrd=[]
for x in range(len(stateAv)):
    stateStd.append([stateSD[x][int(y)] for y in stateOrd[x]])
    
for x in range(len(stateStd)):
    stateStdOrd.append([stateStd[x][y][1] for y in range(len(stateStd[x]))])

In [13]:
#THIS FRAME WILL FIND THE COMMON SET OF GENES EXPRESSED AMONG ALL THE STATES
#find the state with the min # of expressed genes
nonZero=[]
for y in range(len(stateAv)):
    nonZero.append([stateAv[y][x] for x in range(len(stateAv[0])) if stateAv[y][x][1]!=0])
for x in range(len(nonZero)):
    print(len(nonZero[x]))
    
#take only the non-zero expressed genes
stateOrd2=[]
for x in range(len(stateOrd)):
    stateOrd2.append(stateOrd[x][:32215])
#find the genes which are common to all
print("     ")
statesInt=list(set(stateOrd2[0])&set(stateOrd2[1]))
print(len(statesInt))
for x in range(22):
    statesInt=list(set(statesInt)&set(stateOrd2[x+2]))
    print(len(statesInt))

comGeneOrd=[]
for x in range(len(stateOrd2)):
    comGeneOrd.append([y for y in stateOrd2[x] if y in statesInt]) 

44203
39022
44808
40710
38934
34398
43373
39801
38213
37495
42617
39015
37979
40207
42612
43357
40323
37951
32514
43434
38159
36064
41428
32215
     
30179
29499
28909
28402
27340
26889
26642
26071
25858
25727
25528
25407
25264
25231
25210
25142
25089
24752
24731
24651
24519
24478
24035


In [15]:
#take the top targ or each state and take their common genes and run analysis on that
redGeneOrd=[]
off=0
targ=250
for x in range(len(comGeneOrd)):
    redGeneOrd.append(comGeneOrd[x][off:(targ+off)])
redStatesInt=list(set(redGeneOrd[0])&set(redGeneOrd[1]))
for x in range(22):
    redStatesInt=list(set(redStatesInt)&set(redGeneOrd[x+2]))
geneRedFin=[]
for x in range(len(comGeneOrd)):
    geneRedFin.append([x for x in comGeneOrd[x] if x in redStatesInt])

24

In [ ]:
#do a cycle decomposition on the permutations
cycFin=[]
for y in range(len(geneRedFin)):
    cycles=[]
    cyclesTally=[]
    for x in patientOrd:
        p=patientOrd.index(x)
        if p in cyclesTally:
            continue
        else: 
            cycInt=[]
            cycInt.append(p)
            cyclesTally.append(p)
            z=geneRedFin[y].index(x)
            if z==p:
                cycles.append(cycInt)   
            else:    
                for w in range(1000):
                    cycInt.append(z)
                    cyclesTally.append(z)
                    q=patientOrd[z]
                    z=geneRedFin[y].index(q)
                    if z==p: 
                        break
                cycles.append(cycInt)
    cycFin.append(cycles)  
cycLen=[len(cycFin[x]) for x in range(len(cycFin))]